In [1]:
import pandas as pd
import numpy as np
import re

ELEC = '../dataset/congress/pres_by_county.csv'
ZIP = '../dataset/geo/ZIP-COUNTY-FIPS_2017-06.csv'


In [2]:
elec = pd.read_csv(ELEC)
zip = pd.read_csv(ZIP)
elec.head()

,Unnamed: 0,year,state_abbrev,county_name,GEOID,winner,party_win,democrat,republican
0,1,2000,AL,Autauga,1001.0,George W. Bush,republican,28.7,69.7
1,2,2000,AL,Baldwin,1003.0,George W. Bush,republican,24.8,72.4
2,3,2000,AL,Barbour,1005.0,Al Gore,democrat,49.9,49.0
3,4,2000,AL,Bibb,1007.0,George W. Bush,republican,38.2,60.2
4,5,2000,AL,Blount,1009.0,George W. Bush,republican,27.7,70.5


In [3]:
elec['county'] = elec['county_name'].str.lower().str.replace('county', '')
elec['c_name'] = elec.apply(lambda row: f"{row['county']} county, us-{row['state_abbrev'].lower()}", axis=1)
unique_c = elec['c_name'].unique()

unique_c

dict_addr = {
    'addresses': [{"objectid": i, "address": c, "countryCode": "us"} for i, c in enumerate(unique_c)]
}
dict_addr['addresses']

[{'objectid': 0, 'address': 'autauga county, us-al', 'countryCode': 'us'},
 {'objectid': 1, 'address': 'baldwin county, us-al', 'countryCode': 'us'},
 {'objectid': 2, 'address': 'barbour county, us-al', 'countryCode': 'us'},
 {'objectid': 3, 'address': 'bibb county, us-al', 'countryCode': 'us'},
 {'objectid': 4, 'address': 'blount county, us-al', 'countryCode': 'us'},
 {'objectid': 5, 'address': 'bullock county, us-al', 'countryCode': 'us'},
 {'objectid': 6, 'address': 'butler county, us-al', 'countryCode': 'us'},
 {'objectid': 7, 'address': 'calhoun county, us-al', 'countryCode': 'us'},
 {'objectid': 8, 'address': 'chambers county, us-al', 'countryCode': 'us'},
 {'objectid': 9, 'address': 'cherokee county, us-al', 'countryCode': 'us'},
 {'objectid': 10, 'address': 'chilton county, us-al', 'countryCode': 'us'},
 {'objectid': 11, 'address': 'choctaw county, us-al', 'countryCode': 'us'},
 {'objectid': 12, 'address': 'clarke county, us-al', 'countryCode': 'us'},
 {'objectid': 13, 'address

In [15]:

# from zip keep only the first occurrence of STCOUNTYFP
zip = zip.drop_duplicates(subset='STCOUNTYFP', keep='first')
zip['COUNTYNAME'] = zip['COUNTYNAME'].str.replace(' County', '')
elec = elec.merge(zip, left_on=['county_name', 'state_abbrev'], right_on=['COUNTYNAME', 'STATE'], how='left')

def getDistrict(row):
    try:
        # print(row['county_name'])
        district = re.match(r'District (\d+)', row['county_name'])
        if district:
            return district.groups()[0]
    except:
        return np.nan
    return np.nan

elec['district'] = elec.apply(getDistrict, axis=1)
elec[(elec['district'].isna() == True) * (elec['ZIP'].isna())]


,Unnamed: 0,year,state_abbrev,county_name,GEOID,winner,party_win,democrat,republican,ZIP,COUNTYNAME,STATE,STCOUNTYFP,CLASSFP,district
24,25,2000,AL,Dekalb,1049.0,George W. Bush,republican,34.8,63.2,NaN,NaN,NaN,NaN,NaN,NaN
325,326,2000,CT,Statewide Writein,NaN,Al Gore,democrat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
331,332,2000,DC,District Of Columbia,11001.0,Al Gore,democrat,85.2,9.0,NaN,NaN,NaN,NaN,NaN,NaN
344,345,2000,FL,Desoto,12027.0,George W. Bush,republican,42.5,54.4,NaN,NaN,NaN,NaN,NaN,NaN
442,443,2000,GA,Dekalb,13089.0,Al Gore,democrat,70.2,26.7,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18748,18749,2020,VA,Waynesboro City,51820.0,Donald J Trump,republican,46.3,51.4,NaN,NaN,NaN,NaN,NaN,NaN
18750,18751,2020,VA,Williamsburg City,51830.0,Joseph R Biden Jr,democrat,69.5,28.5,NaN,NaN,NaN,NaN,NaN,NaN
18751,18752,2020,VA,Winchester City,51840.0,Joseph R Biden Jr,democrat,54.6,43.1,NaN,NaN,NaN,NaN,NaN,NaN
18820,18821,2020,WV,Mcdowell,54047.0,Donald J Trump,republican,20.4,78.9,NaN,NaN,NaN,NaN,NaN,NaN
